# Pynanolib Manual

Welcome to Pynanolib project. This is a small library I decided to create for parsing, processing, visualizing data files obtained by performing experiment with a [Nanonis SPM Control System TM](https://www.specs-group.com/nanonis/products/). Such a software is used among experimental physicists in laboratories dedicated to electron microscopy and spectroscopy. Despite the purpose of such a project being quite specific, the approach and idea presented inside this project are applicable to any kind of data file sets, and on this base, they can be extended to any interested data files. Before going forward and explain the content of Pynanolib, let me reference two other works helped me to developing Pynanolib. The former is the library [Nanolib](https://github.com/ethz-micro/matlab_nanonis) written in MATLAB. This was a work I contributed to, several years ago. The latter is the library [nanonispy](https://github.com/underchemist/nanonispy), written in python to parse nanonis data files.

## Pynanolib modules

Pynanolib contained three modules: load, process and plot, respectively in load.py, process.py and plot.py files.

## Load:

The load module reads a "nanonis" file (files with extension .sxm or .dat). It creates and stores the information of such a file in an istance of the Nanonisfile class that has simply three attributes and no public methods:
- __load.Nanonisfile.metadata__ is a dictionary storing the file metadata, e.g. file name, file path, file size, file data creation etc.
- __load.Nanonisfile.header__ is a dictionary storing the file header, i.e. the information provided by Nanonis software regarding the experiment done, e.g. experiment name, integration time, number of points, comments, etc. Any information describing the experiment preceding the data are contained in the header.
- __load.Nanonisfile.data__ is a dictionary storing the corresponding channel name to its data as numpy array.

## Process:

The process module contains the following methods: _processSingleFile_ and _processMultipleFiles_.
- __process.processSingleFile__ can process one or more desired data arrays from the Nanonisfile.data dictionary of a single NanonisFile object. Such a method applies one of the processes listed in the global variable "processAllowed" defined in process.py. Some of this processes need a reference data, some others not. For example, considering a channel "Voltage" and a reference channel "Current" in the load.Nanonisfile.data. By using the process: "Division", we can obtain a processed channel "Resistence" as a result(Ohm's law: R = V/I). The resulting data will be added to the load.Nanonisfile.data dictionary under the key named "Voltage Division by Current". Examples of processes that do not need a reference data are the "Min-MaxNormalization", "Standardization" and "Differantiation".
- __process.processMultiFiles__ can process one or more desired data arrays from several NanonisFile objects, but it has to be implemented yet. ;)

## Plot:

The plot module contains the following methods: plot.plotSingleFile.
- __plot.plotSingleFile__ can plot one or more desired data arrays from the Nanonisfile.data dictionary of a single NanonisFile object. It can plot all the desired data in one couple of axes, in several axes within a single figure, or in _different figures (this has to be implemented yet.)_
- __plot.plotMultiFiles__ can plot one or more desired data arrays from several NanonisFile objects, but it has to be implemented yet. ;)

## Examples
